In [ ]:
using RCall
using BenchmarkTools
using CSV
using DataFrames
using GLM
using PenalizedGLMM
using SnpArrays

In [ ]:
pkgversion(PenalizedGLMM)

In [ ]:
pkgversion(BenchmarkTools)

In [ ]:
Threads.nthreads()

In [ ]:
cd(joinpath("..", ".."))

In [ ]:
### data filepaths ###
const covfile = joinpath("data", "penncath.csv") # includes binary phenotype data
const plinkfile = joinpath("data", "qc_penncath")
const grmfile = joinpath("results", "penncath-k.csv.gz") # genetic relatedness matrix

In [ ]:
### read in covariate file ###
covdf = CSV.read(covfile, DataFrame)
#fam = CSV.read(famfile, DataFrame, delim = " ", header = false)
#rename!(fam,:Column1 => :FamID)
#covdf = rightjoin(covdf, fam, on = :FamID)

In [ ]:
# force CAD to be a float (for normal approx)
covdf[!,:CADfloat] = convert(Array{Float64}, covdf[!,:CAD])
CSV.write(joinpath("results", "penncath-float.csv"), covdf)
const floatcovfile = joinpath("results", "penncath-float.csv")

In [ ]:
nullmodel_normal = pglmm_null(@formula(CADfloat ~ sex + age), 
    covfile = floatcovfile, 
    grmfile = grmfile, 
    family = Normal(), 
    link = IdentityLink())

In [ ]:
nullmodel_normal.τ

In [ ]:
# @elapsed pglmm_null(@formula(CADfloat ~ sex + age), covfile = floatcovfile, grmfile = grmfile, 
#     family = Normal(), link = IdentityLink())

In [ ]:
print(nullmodel_normal.converged)

In [ ]:
@elapsed pglmm(nullmodel_normal, plinkfile)

In [ ]:
#### Prediction Objective ####

In [ ]:
# training / testing split - exported from plmmr fit
traininds = CSV.read(joinpath("results", "train_indices.csv"), DataFrame)
traininds = traininds[:,1]
testinds = CSV.read(joinpath("results", "test_indices.csv"), DataFrame)
testinds = testinds[:,1]

In [ ]:
nullmodel_normal = pglmm_null(@formula(CADfloat ~ sex + age), 
    family = Normal(),
    link = IdentityLink(),
    covfile = floatcovfile, 
    grmfile = grmfile, 
    covrowinds = traininds,
    grminds = traininds)

In [ ]:
modelfit_normal = pglmm(nullmodel_normal, 
    plinkfile, 
    geneticrowinds = traininds,
    penalty_factor_X = Float64[0; ones(2)]) # we are penalizing the covariates

In [ ]:
using JLD2

@save joinpath("results", "PennCath_pglmm_train.jld2") modelfit_normal
# modelfile = jldopen(joinpath("results", "PennCath_pglmm_train.jld2"), "r")
# modelfit_normal = modelfile["modelfit_normal"]
# close(modelfile)

In [ ]:
pglmmBIC = PenalizedGLMM.GIC(modelfit_normal, :BIC)

In [ ]:
modelfit_normal.alphas[:, pglmmBIC]

In [ ]:
modelfit_normal.betas[:, pglmmBIC]

In [ ]:
yhat = PenalizedGLMM.predict(modelfit_normal,
                            @formula(CADfloat ~ sex + age),
                            floatcovfile,
                            plinkfile,
                            grmfile = grmfile,
                            testrowinds = testinds,
                            trainrowinds = traininds,
                            geneticrowinds = testinds,
                            s = [pglmmBIC],
                            outtype = :response)
yhat = vec(yhat)

In [ ]:
loss = (yhat - covdf[testinds, :CADfloat]).^2
mean(loss)